# **NFT COLLABOT**

> Summary Value Prop Blog

*NFT CollaBot* is a project that designed by obtaining requirements of NFT ecosystem. NFT marketplace from Tezos ecosystem named objkt.com will be starting point of the project. NFT CollaBot aims to make underrated artists more visible.

In [1]:
# importing libraries for data process
import pandas as pd
import numpy as np

# importing libraries to handle web scraping
import requests
import json

# string operations will be handled during data analysis
import re
import string
from io import StringIO

# also using these libraries
import random
import time

### API DOCUMENTATION

Objktcom has a published API documentation
*https://data.objkt.com/docs/#listing*

Documentation points GraphiQL explore page to try queries.
There is a link below that explains how to write queries with GraphiQL
*https://graphql.org/learn/queries/*

Besides, there is a useful link to code on Python using GraphiQL queries. Check the following link:
*https://towardsdatascience.com/connecting-to-a-graphql-api-using-python-246dda927840*

In [2]:
# objktcom api endpoint will be used for several times to evaluate queries

api_endpoint = 'https://data.objkt.com/v2/graphql'

> ##### GRAPHIQL
GraphiQL playground is useful tool to understand insight of the query format
>  https://data.objkt.com/explore/

> #### Retrieve Beginning Primary Key Value

There are thousands of NFTs belongs to a tag. It will take too long to get all of the data. Besides, it will be more effective to show newer NFTs to achieve main target of the project.

The mechanism have to use token_pk to request data, according to API constrains. Decided to choose a token_pk using a timestamp.

In [3]:
# query for getting primary key has evaluated below:

#initial_token_pk_query="""query {
#    token(where: {timestamp: {_gt: "2022-06-15T00:12:00+00:00"}}, limit: 1) {
#    creators {
#        token_pk
#    }
#    }
#}"""

"""
# send request
token_pk_data = requests.post(api_endpoint, json={'query': initial_token_pk_query})
token_pk_data = json.loads(token_pk_data.text)
token_pk_data=token_pk_data['data']['token']

# output is 8682680
# this primary key will be used in the next steps of the development
"""

"\n# send request\ntoken_pk_data = requests.post(api_endpoint, json={'query': initial_token_pk_query})\ntoken_pk_data = json.loads(token_pk_data.text)\ntoken_pk_data=token_pk_data['data']['token']\n\n# output is 8682680\n# this primary key will be used in the next steps of the development\n"

### RECURSIVE MECHANISM TO GET TOKEN PRIMARY KEY DATA

After getting primary key of the starting point, query for tag of token will be evaluated. In this query, only primary keys of tokens will be requested from API. Then, this primary key value will be used to get information about related NFT. This mechanism aims to implement request process recursively.

In [4]:
counter=[0]      # counts how many times has the function is executed

def request_nft_token_pk(tag_name_variable):
      if counter[0]>0:global pk_value
      if counter[0]==0: pk_value=8682680

      # declare query to request data from api
      nftToken_pk_query="""query {
      token_tag(where: {tag: {name: {_eq: tag_name_variable}}, _and: {token_pk: {_gt: last_pk_value}}}) {
      token_pk
      }
      }"""
      #create dynamic query to request recursively
      nftToken_pk_query=nftToken_pk_query.replace("tag_name_variable",tag_name_variable)
      nftToken_pk_query=nftToken_pk_query.replace("last_pk_value",str(pk_value))

      #request data from objkt.com api endpoint
      token_pk_request = requests.post(api_endpoint, json={'query': nftToken_pk_query})
      token_pk_request = json.loads(token_pk_request.text)
      token_pk_request=token_pk_request['data']['token_tag']

      #have to assign a boolean variable to check is response empty or not
      isEmpty_response=False

      if counter[0]>0:                       # to avoid unbound local error...
            global primaryKey_df             # ...declare as global
            global primaryKey_df_loop

      if counter[0]==0 :
            primaryKey_df = pd.DataFrame()
            primaryKey_df_loop = pd.DataFrame()
      primaryKey_df_loop = pd.DataFrame(token_pk_request)            # convert json data into a data frame
      primaryKey_df=pd.concat([primaryKey_df,primaryKey_df_loop])    # concatenate two data frames to save all data in one data frame


      if primaryKey_df_loop.shape[0]>0:         # assign latest primary key value to a new variable to use this value for recursion queries
            pk_value=int(primaryKey_df_loop.loc[(primaryKey_df_loop.shape[0])-1].values)
      else: isEmpty_response=True               # make boolean value True whenever response is empty

      # increase one the variable for each execution of recursive function
      counter[0]+=1

      # create recursive algorithm inside function
      if not isEmpty_response:     # execute the function since it does not retrieve any data from api endpoint
            #time.sleep(0.1)
            return request_nft_token_pk(tag_name_variable)

request_nft_token_pk("cartoon")

# arrange index of the data frame
primaryKey_df.set_index(np.arange(primaryKey_df.shape[0]))

,token_pk
0,8858475
1,8857749
2,8861400
3,8857945
4,8867693
...,...
12230,32744980
12231,32748270
12232,32759340
12233,32761655


### NFT Recommendation Mechanism

> **Random Recommendation**

Select an NFT randomly to perform functionality more fair for everyone

> **Future Aspects:**

On the other hand, this mechanism will be extended with an algorithm in further stages

There will be an input option to find underrated artists on ecosystem. The algorithm will recommend artists with low sales

In [27]:
import random
randomNFT_index=random.sample(range(primaryKey_df.shape[0]),10)
randomNFT=primaryKey_df.iloc[randomNFT_index[0]]

print(randomNFT['token_pk'])

9969699


#### Get NFT Info From API

In [34]:
#query must be evaluated for the chosen primary key

nft_info_query= """query{
  token(where: {pk: {_eq: "random_pk_value"}}) {
    name
    token_id
    mime
    supply
    artifact_uri
    holders(where: {holder_address: {_neq: "tz1burnburnburnburnburnburnburjAYjjX"}}) {
      quantity
      holder_address
    }
    asks(where: {status: {_eq: "active"}}) {
      price
      seller_address
      amount_left
      amount
    }
    fa {
      token_link
    }
  }
  token_creator(where: {token_pk: {_eq: "random_pk_value"}}) {
    creator_address
  }
}
"""

nft_info_query=nft_info_query.replace("random_pk_value",str(randomNFT['token_pk']))

In [35]:
nft_info_request = requests.post(api_endpoint, json={'query': nft_info_query})
nft_info_request = json.loads(nft_info_request.text)
nft_info_request = nft_info_request['data']

In [37]:
nft_info_request

{'token': [{'name': 'wwyg🐸cards #6',
   'token_id': '288972',
   'mime': 'application/x-directory',
   'supply': 1,
   'artifact_uri': 'ipfs://QmUqFXz6mPP7gFVjyicR2FwngUefaahx4rS4sd9kcaRn4N?fxhash=op6DEmYUGoqK5GsLdfzS29ysDGJYhExszmem91rvBTXzNC1ztqR',
   'holders': [{'quantity': 0,
     'holder_address': 'tz2RpKQUSUygQ2HaSGfzPptumV1Uk8mfBKaP'},
    {'quantity': 0, 'holder_address': 'KT1Xo5B7PNBAeynZPmca4bRh6LQow4og1Zb9'},
    {'quantity': 1, 'holder_address': 'tz1YQs3qHgQGGQm14vX8vMDWwzY1DFXHCzJB'}],
   'asks': [],
   'fa': {'token_link': 'https://fxhash.xyz/gentk/:id'}}],
 'token_creator': [{'creator_address': 'tz1QR96zd73MEEQT2TSp1FSfm1Pc8kZvBkc9'}]}

#### TzKT API

TzKT API has an endpoint to get data of wallets on tezos ecosystem. Endpoint link:
>https://api.tzkt.io/

In the following stage, have to match creator address with the tezos ecosystem registered name

In [ ]:
# get creator address' name from tzkt.io API

wallet_address=requested_nft_df['creators'][randomNFT]               # assign the wallet address
wallet_address=wallet_address[1::]                                   # there ia an empty space on 0th character, to write a proper url

account_data_url=f"https://api.tzkt.io/v1/accounts/{wallet_address}"        # tzkt.io API endpoint
response = requests.get(account_data_url)                                   # send request to get data

try:
    response=response.json()
    print("Owner of the wallet address is "+response['alias'])       # printing username of account owner to see function is whether working or not
except KeyError:                                                     # avoid error if the user has not registered with a name
    print("The user has no name on TzKT API")                        #'alias' attribute does not exist if the user has not registered with a name
except json.decoder.JSONDecodeError:                                 # avoid json-related errors
    pass

In [ ]:
account_metadata_url=f"https://api.tzkt.io/v1/accounts/{wallet_address}/metadata"        # tzkt.io API endpoint
response = requests.get(account_metadata_url)                                            # send request to get data

try:response=response.json()                                                             # make object type subscriptable to implement next process
except json.decoder.JSONDecodeError:pass                                                 # avoid from exceptional json error

In [ ]:
# get instagram username from metadata, if exists on metadata
try:
    if(len(response['instagram'])):
        print("https://www.instagram.com/"+response['instagram']+"/")
except json.decoder.JSONDecodeError:
    pass
except KeyError or TypeError:                                 # program gets KeyError when relevant metadata does not available for the user
    print("instagram is null")                   # print to test null case, only for validating the code


# get twitter username from metadata, if exists on metadata
try:
    if(len(response['twitter'])):
        print("https://www.twitter.com/"+response['twitter'])
except json.decoder.JSONDecodeError:
    pass
except KeyError or TypeError:
    print("Twitter is null")

In [ ]:
# merge all the related cells into a function to make it more dynamic
"""
def artist_info(x):
    wallet_address=requested_nft_df['creators'][x]                       # assign the wallet address
    wallet_address=wallet_address[1::]                                   # there ia an empty space on 0th character, to write a proper url

    account_data_url=f"https://api.tzkt.io/v1/accounts/{wallet_address}"        # tzkt.io API endpoint
    response = requests.get(account_data_url)                                   # send request to get data

    try:
        response=response.json()
        print("Owner of the wallet address is "+response['alias'])       # printing username of account owner to see function is whether working or not
    except KeyError or json.decoder.JSONDecodeError:pass

    account_metadata_url=f"https://api.tzkt.io/v1/accounts/{wallet_address}/metadata"        # tzkt.io API endpoint
    response = requests.get(account_metadata_url)                                            # send request to get data

    try:
        response=response.json()
        if(len(response['instagram'])):
            print("https://www.instagram.com/"+response['instagram']+"/")
    except json.decoder.JSONDecodeError:pass
    except KeyError:                                 # program gets KeyError when relevant metadata does not available for the user
        print("instagram is null")                   # print to test null case, only for validating the code


    # get twitter username from metadata, if exists on metadata
    try:
        response=response.json()
        if(len(response['twitter'])):
            print("https://www.twitter.com/"+response['twitter'])
    except json.decoder.JSONDecodeError:pass
    except AttributeError:
        if(len(response['twitter'])):
            print("https://www.twitter.com/"+response['twitter'])
    except KeyError:print("Twitter is null")


import random
randomNFT_index=random.sample(range(500), 10)
randomNFT=randomNFT_index[0]

artist_info(randomNFT)"""
